### Import

##### (import 안될 경우) 실행에 필요한 라이브러리 설치바람

In [1]:
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
#from selenium.webdriver.chrome.options import Options
import numpy as np
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import requests
#import html5lib
import urllib3
import os
import openpyxl

In [75]:
# 해쉬 생성용 라이브러리
import hashlib as hlb

In [76]:
# 유의사항 : 크롬 브라우저로 데이터 수집을 잘 못할 경우, 파이어폭스 브라우저 사용도 시도해 보기 바람.
from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.firefox.options import Options

In [77]:
# DISPLAY 글자 수 늘리기 : 데이터프레임 열람시 활용 가능 (40 <-> 4000)
#pd.options.display.max_colwidth = 4000

In [78]:
# 라이브러리 즉석 설치가 필요할 경우 해당 커맨드 사용 권장
#!pip install selenium

### Config (new, old)

##### 유의사항 : CSS 가 수집에 유리할지 XPATH 가 수집에 유리할지는 게시판마다 다름 ###

In [79]:
### CONFIG 1 ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
# 데이터 수집을 시작할 페이지 번호
start_page = 1
# 마지막 페이지 번호
end_page = 2073
# 마지막 페이지 글 수
num_a_fin = 2
# 한 페이지에 보여지는 글 개수
cnt_a = 10
### CONFIG 2 ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
### 시작 url
url_str_ = 'https://www.sejong.go.kr/bbs/R0079/list.do?pageIndex={}'
# (게시판 글 리스트에서의) 글 제목 (XPATH OR CSS)
# 유의사항 : 숫자가 1부터 +1씩 순차적 증가가 아닐 경우, 밑의 실행 코드 부분을 수정해야 함.
article_attr = '//*[@id="txt"]/div/div[2]/table/tbody/tr[{}]/td[2]/a'
### CONFIG 3 ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
# 유의사항 : XPATH일 때, 그 속성값 수가 2개 초과일 경우 CSS로 바꾸는 것을 추천.
# 글 제목 (XPATH OR CSS)
ttl_attr1 = r'//*[@id="txt"]/div/div[1]/div[1]/h2'
#ttl_attr2 = r'//*[@id="txt"]/div/div[1]/div[1]/h2'
# 글 작성일 (XPATH OR CSS)
dt_attr1 = r'//*[@id="txt"]/div/div[1]/div[1]/span[3]'
#dt_attr2 = r'//*[@id="txt"]/div/div[1]/div[1]/span[3]'
# 글 내용 ((태그가) 두 개인 경우 cntnt_attr2  주석 풀고 추가 입력) (XPATH OR CSS)
cntnt_attr1 = r'//*[@id="txt"]/div/div[1]/div[3]'
#cntnt_attr2 = r'//*[@id="txt"]/div/div[1]/div[3]'
# 저작권 ((태그가) 두 개인 경우 cprt_attr2 주석 풀고 추가 입력) (XPATH OR CSS)
cprt_attr1 = r'//*[@id="txt"]/div/div[1]/div[4]/div/img'
# cprt_attr2 = r'//*[@id="txt"]/div/div[1]/div[3]/div/img'
### CONFIG 4 ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
# 'CN_LINK_ADDR' 태그 # 참고 : ':nth-child(1)' 등 지워야 함  
# 유의사항 : 전체 태그일 경우, 태그 앞에 '#' 붙어야 함
# 전체 태그 예시 : '#txt > div > div.no-more-tables > table > tbody > tr > td.subject > a'
# 유의사항 : 전체 태그로 해당 값을 찾지 못할 경우, 부분 태그를 사용해야 하고 이 때는 '#'을 빼야함 
# 부분 태그 예시 : 'tr > td.subject > a'
# 이 부분은 CSS 태그 값만 (CSS ONLY)
### ---------------------------------------------------------------------------------------------------------------
pcla_css = '#txt > div > div.no-more-tables > table > tbody > tr > td.subject > a'
### CONFIG 5 ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
# 최종 생성 파일에는 입력되지 않으나, 파일명 등의 구분을 위해 크롤러 번호를 입력
bbs_sn_num = 37388
### ---------------------------------------------------------------------------------------------------------------
### ---------------------------------------------------------------------------------------------------------------
### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
### 이 밑은 CONFIG 하지 말기 (재실행의 용이성을 위해 이곳에 적어놓음)
started_tm = datetime.now().strftime('%Y%m%d%H%M%S')
folder_nm = 'crawler_{}_{}'.format(bbs_sn_num, started_tm)
dt_format = '%Y-%m-%d %H:%M:%S'
# 총 페이지 계산시 나머지
if num_a_fin == 0:
    rmd = 0
else:
    rmd = 1
# 모을 글의 총 페이지 개수
tot_page = (end_page - start_page) + rmd
### Browsing 여부 ### ### ### ### ### ### ### ### ### ### ### 
options = Options()
# 이미지 로딩 OFF (속도 개선)
options.add_argument('--blink-settings=imagesEnabled=false')
# 스크롤 다운 필요한 게시판의 경우 필요 # headless 상태이면 불필요
# 게시판 크기 축소 (특정 게시판의 경우, 클릭 용이함을 위해)
#options.add_argument("force-device-scale-factor=0.25")
# 브라우저 크기 조절
#options.add_argument("--window-size=1600,1000")
# Headless 여부
options.add_argument('-headless')

In [80]:
#contents_body > div.board_list > table > tbody > tr:nth-child(1) > td.txt_left > a

### Config (click-button)

In [ ]:
# ### CONFIG 1 ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
# # 데이터 수집을 시작할 페이지 번호
# start_page = 1
# # 마지막 페이지 번호
# end_page = 55
# # 마지막 페이지 글 수
# num_a_fin = 3
# # 한 페이지에 보여지는 글 개수
# cnt_a = 12
# if num_a_fin == 0:
#     rmd = 0
# else:
#     rmd = 1
# # 모을 글의 총 페이지 개수
# tot_page = (end_page - start_page) + rmd
# # 게시판 하단에 표시할 페이지 개수
# page_cnt = 10
# # 현재 페이지 (0 ~ 9)
# cnt_cp = int(start_page % page_cnt)
# ### ---------------------------------------------------------------------------------------------------------------
# ### CONFIG 2 ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
# ### 시작 url
# url_input = 'https://www.sangju.go.kr/archive/historicalRecords/index.do?menu_idx=103&main_case=1&keywd='
# # (게시판 글 리스트에서의) 글 제목
# article_attr1 = '#boardItem > a:nth-child({}) > div > p'
# article_attr2 = '//*[@id="boardItem"]/a[{}]/div/p'
# # 다음 페이지 이동 버튼
# nxt_btn_attr = '#board_paging > a.maxPagination_next'
# # 현재 페이지 버튼
# cpage_attr = '#board_paging > a:nth-child({})'
# ### CONFIG 3 ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
# # 글 제목
# ttl_attr1 = '#subPage > div.printPage > div.archive-view > div > div.archive-view__collection-title'
# ttl_attr2 = '#subPage > div.printPage > div.archive-view > div.archive-view__title'
# # 글 작성일
# dt_attr1 = '#subPage > div.printPage > div.archive-view > div.archive-view__table > table > tbody > tr:nth-child(3) > td:nth-child(2)'
# dt_attr2 = '#subPage > div.printPage > div.archive-view > div.archive-view__table > table > tbody > tr:nth-child(3) > td:nth-child(2)'
# # 글 내용
# cntnt_attr1 = '#subPage > div.printPage > div.archive-view > div > div.archive-view__collection-text'
# cntnt_attr2 = '#subPage > div.printPage > div.archive-view > div.archive-view__table > table > tbody > tr:nth-child(4) > td'
# # 파일 이름
# fnm_attr = r'//*[@id="contents_body"]/article/div[2]'
# # 저작권
# cprt_attr1 = '#subPage > div.printPage > div.archive-view > div.archive-view__table > table > tbody > tr:nth-child(5) > td > a > img'
# ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
# ### CONFIG 4 ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
# # 'CN_LINK_ADDR' 태그 # 참고 : ':nth-child(1)' 등 지워야 함  
# # 유의사항 : 전체 태그일 경우, 태그 앞에 '#' 붙어야 함
# # 전체 태그 예시 : '#txt > div > div.no-more-tables > table > tbody > tr > td.subject > a'
# # 유의사항 : 전체 태그로 해당 값을 찾지 못할 경우, 부분 태그를 사용해야 하고 이 때는 '#'을 빼야함 
# # 부분 태그 예시 : 'tr > td.subject > a'
# # 이 부분은 CSS 태그 값만 (CSS ONLY)
# ### ---------------------------------------------------------------------------------------------------------------
# pcla_css = '#txt > div > div.no-more-tables > table > tbody > tr > td.subject > a'
# ### CONFIG 5 ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
# # 최종 생성 파일에는 입력되지 않으나, 파일명 등의 구분을 위해 크롤러 번호를 입력
# bbs_sn_num = 37388
# ### ---------------------------------------------------------------------------------------------------------------
# ### ---------------------------------------------------------------------------------------------------------------
# ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
# ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
# ### 이 밑은 CONFIG 하지 말기 (재실행의 용이성을 위해 이곳에 적어놓음)
# started_tm = datetime.now().strftime('%Y%m%d%H%M%S')
# folder_nm = 'crawler_{}_{}'.format(bbs_sn_num, started_tm)
# dt_format = '%Y-%m-%d %H:%M:%S'
# ### Browsing 여부
# options = Options()
# # 이미지 로딩 OFF (속도 개선)
# options.add_argument('--blink-settings=imagesEnabled=false')
# #options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
# # options.add_argument("--window-size=1600,1000")
# #options.add_argument('headless')
# #options.add_argument("force-device-scale-factor=0.05")

### Def

In [81]:
### OLD
### 전처리 함수
def preprocess(df_input,bbs_sn):
    ### CONFIG ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
    fp_addr = ''
    cprt_type = ['img_opentype01', 'img_opentype02', 'img_opentype03', 'img_opentype04']
    ptn_pdi1 = ' [0-9]{2}\:[0-9]{2}\:[0-9]{2}'
    ptn_pdi2 = ' [0-9]{2}\:[0-9]{2}'
    ptn_fnm_adr = '\([0-9.]{0,10}[KB|B|MB|GB]{1,2} \/ 다운로드 .+\/ .+\)'
    ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###

    # 작성일 처리
    for i in range(len(df_input)):
        df_input['DT_INFO'].iloc[i] = df_input['DT_INFO'].iloc[i].replace('작성일', '')
        # hh:mm 삭제
        hhmm = re.findall(ptn_pdi1, df_input['DT_INFO'].iloc[i])
        if len(hhmm) != 0:
            df_input['DT_INFO'].iloc[i] = df_input['DT_INFO'].iloc[i].replace(hhmm[0], '')
        hhmm2 = re.findall(ptn_pdi2, df_input['DT_INFO'].iloc[i])
        if len(hhmm2) != 0:
            df_input['DT_INFO'].iloc[i] = df_input['DT_INFO'].iloc[i].replace(hhmm2[0], '')
        # yyyy/mm/dd -> yyyymmdd
        df_input['DT_INFO'].iloc[i] = df_input['DT_INFO'].iloc[i].replace('/', '')
        # yyyy-mm-dd -> yyyymmdd
        df_input['DT_INFO'].iloc[i] = df_input['DT_INFO'].iloc[i].replace('-', '')
        # yyyy.mm.dd -> yyyymmdd
        df_input['DT_INFO'].iloc[i] = df_input['DT_INFO'].iloc[i].replace('.', '')

    ## 파일명 처리
    df_input['FILE_NM'] = df_input['FILE_NM'].fillna('')
    for i in range(len(df_input)):
        df_input['FILE_NM'].iloc[i] = df_input['FILE_NM'].iloc[i].replace('다운로드 미리보기', '')
        df_input['FILE_NM'].iloc[i] = df_input['FILE_NM'].iloc[i].replace('첨부파일', '')

        prnths = re.findall(ptn_fnm_adr, df_input['FILE_NM'].iloc[i])
        if len(prnths) != 0:
            df_input['FILE_NM'].iloc[i] = df_input['FILE_NM'].iloc[i].replace(prnths[0], '')
            df_input['FILE_NM'].iloc[i] = df_input['FILE_NM'].iloc[i].lstrip('\n')
            df_input['FILE_NM'].iloc[i] = df_input['FILE_NM'].iloc[i].rstrip(' \n')
            df_input['FILE_NM'].iloc[i] = df_input['FILE_NM'].iloc[i].rstrip('\n')
            # hwp 파일 두 개 이상(추후 확장자 추가 바람)
            if df_input['FILE_NM'].iloc[i].count('.hwp') > 1:
                df_input['FILE_NM'].iloc[i] = df_input['FILE_NM'].iloc[i].replace('\n', '||')
                df_input['FILE_NM'].iloc[i] = df_input['FILE_NM'].iloc[i].rstrip('||')
    ## 파일 스토리지 경로 입력
    df_input['FILE_PATH'] = fp_addr
    for i in range(len(df_input)):
        if df_input['FILE_NM'].iloc[i] == '':
            df_input['FILE_NM'].iloc[i] = None
            df_input['FILE_PATH'].iloc[i] = None
    ## 그외 파일 관련 칼럼 생성 및 입력
    df_input['FILE_PATH_CN'] = None
    df_input['FILE_NM_CN'] = None

    # 저작권
    df_input['CP_TYPE_NM'] = ''
    for i in range(len(df_input)):
        if cprt_type[0] in df_input['CP_TYPE_NM_'].iloc[i]:
            df_input.loc[i, 'CP_TYPE_NM'] = '1'
        elif cprt_type[1] in df_input['CP_TYPE_NM_'].iloc[i]:
            df_input.loc[i, 'CP_TYPE_NM'] = '2'
        elif cprt_type[2] in df_input['CP_TYPE_NM_'].iloc[i]:
            df_input.loc[i, 'CP_TYPE_NM'] = '3'
        elif cprt_type[3] in df_input['CP_TYPE_NM_'].iloc[i]:
            df_input.loc[i, 'CP_TYPE_NM'] = '4'
        else:
            df_input.loc[i, 'CP_TYPE_NM'] = '5'
    df_input.drop('CP_TYPE_NM_', inplace=True, axis=1)

    # 시간 입력
    df_input['CRT_DT'] = ''
    df_input['MDFCN_DT'] = ''
    df_input['LAST_MDFCN_DT'] = ''
    for i in range(len(df_input)):
        df_input.loc[i, 'CRT_DT'] = df_input['tm_stamp1'].iloc[i]
        df_input.loc[i, 'MDFCN_DT'] = df_input['tm_stamp2'].iloc[i]
        df_input.loc[i, 'LAST_MDFCN_DT'] = df_input['tm_stamp2'].iloc[i]
    df_input.drop(['tm_stamp1', 'tm_stamp2'], axis=1, inplace=True)

    # URL
    df_input['ACTL_URL_ADDR'] = df_input['URL_ADDR']

    # 이미지 관련 칼럼
    df_input['IMG_PATH'] = None
    df_input['IMG_NM'] = None

    # 링크 관련 칼럼
    df_input['FAIL_LINK_RMRK_CN'] = None
    df_input['FAIL_LINK_STTS_SN'] = None
    df_input['FAIL_LINK_LAST_MDFCN_DT'] = None

    # 그 외 칼럼
    df_input['STTS_SN'] = 200
    df_input['CLCT_SN'] = None
    df_input['RMRK_CN'] = None

    # 크롤러 번호
    df_input['BBS_SN'] = bbs_sn
    
# 해쉬 함수('UNQ_VL_INFO')
def puvi_prprc(df_input):
    df_input['UNQ_VL_INFO'] = ''
    for i in range(len(df_input)):
        # 간혹 'TTL' 획득시 nan 값 존재해서 각각 string 처리함
        sha_str = str(df_input['TTL'].iloc[i]) + str(df_input['URL_ADDR'].iloc[i]) + str(df_input['CN_LINK_ADDR'].iloc[i])
        sha_str_rst = hlb.sha512(sha_str.encode('utf-8')).hexdigest()
        df_input.loc[i,'UNQ_VL_INFO'] = sha_str_rst
    return df_input

# table 칼럼 순서로 맞추기
def col_organizer(df_input):
    if len(df_input.columns) == 24:
        df_out = df_input[['UNQ_VL_INFO', 'CN', 'FILE_PATH_CN', 'FILE_NM_CN',
                        'RMRK_CN', 'FAIL_LINK_RMRK_CN', 'TTL', 'URL_ADDR',
                        'CN_LINK_ADDR', 'ACTL_URL_ADDR', 'DT_INFO', 'IMG_PATH',
                        'IMG_NM', 'FILE_PATH', 'FILE_NM', 'CP_TYPE_NM',
                        'STTS_SN', 'LAST_MDFCN_DT', 'FAIL_LINK_STTS_SN',
                        'FAIL_LINK_LAST_MDFCN_DT', 'CLCT_SN', 'CRT_DT', 'MDFCN_DT', 'BBS_SN']]
    elif len(df_input.columns) == 11:
        df_out = df_input[['UNQ_VL_INFO', 'TTL', 'LINK_ADDR', 'CN_LINK_ADDR',
                           'DT_INFO', 'STTS_INFO', 'CLCT_SN', 'CRT_DT', 'MDFCN_DT',
                           'BBS_SN', 'STTS_DT']]

    return df_out

# 'CLCT_INFO_DTL' 생성함수
def ci_dtl_maker(df_input):
    df_out = df_input[['UNQ_VL_INFO', 'TTL', 'URL_ADDR', 'CN_LINK_ADDR',
                                                  'DT_INFO', 'CRT_DT', 'MDFCN_DT',
                                                  'BBS_SN', 'LAST_MDFCN_DT']]
    df_out = df_out.rename(
        columns={"URL_ADDR": "LINK_ADDR", "LAST_MDFCN_DT": "STTS_DT"})
    df_out.loc[:, 'STTS_INFO'] = 'DONE'
    df_out['SN'] = np.NaN

    return df_out

In [1]:
# 게시판 마다 날짜 패턴이 다를 수 있으므로 추후 여기에 없는 패턴 발견시 추가 바람.
# yyyy년 mm월 dd일(dow) -> yyyymmdd
def pdi_reformer(df_input):
    pattern_y = '[0-9]{4}년'
    pattern_m = '[0-9]{1,2}월'
    pattern_d = '[0-9]{1,2}일'
    for i in range(len(df_input)):
        str_y = re.findall(pattern_y, df_input['ENROLL_DATE'].iloc[i])
        str_m = re.findall(pattern_m, df_input['ENROLL_DATE'].iloc[i])
        str_d = re.findall(pattern_d, df_input['ENROLL_DATE'].iloc[i])
        if len(str_y) != 0 and len(str_m) != 0 and len(str_d) != 0:
            str_y = str_y[0].replace('년','')
            str_m = str_m[0].replace('월','')
            str_d = str_d[0].replace('일','')
            # yyyy d m
            if len(str_m) == 1 and len(str_d) == 1:
                str_comb = str_y + '0' + str_m + '0' + str_d
            # yyyy dd m
            elif len(str_m) == 2 and len(str_d) == 1:
                str_comb = str_y + str_m + '0' + str_d
            # yyyy d mm
            elif len(str_m) == 1 and len(str_d) == 2:
                str_comb = str_y + '0' + str_m + str_d
            # yyyy dd mm
            elif len(str_m) == 2 and len(str_d) == 2:
                str_comb = str_y + str_m + str_d
            # ------------------------------------------
            df_input.loc[i,'ENROLL_DATE'] = str_comb

            
# 속성값 차이에 따라 '작성일 ' 등의 문구가 추가될 수 있으므로, 
# 기존 전처리 함수에서 따로 기능을 분리하게 됨.
# yyyy.mm.dd -> yyyymmdd
# 이 코드 작성 시점까지 yyyy.m.d 등의 패턴은 발견되지 않음.
def pdi_reformer2(df_input):
    pattern_ymd2 = '[0-9]{4}\.[0-9]{2}\.[0-9]{2}'
    str_ymd2 = re.findall(pattern_ymd2, df_input['ENROLL_DATE'].iloc[i])
    if len(str_ymd2) != 0:
        str_ymd2_ = str_ymd2[0].replace('.','')
        df_input.loc[i,'ENROLL_DATE'] = str_ymd2_


# yyyy-mm-dd -> yyyymmdd
# 이 코드 작성 시점까지 yyyy-m-d 등의 패턴은 발견되지 않음.
def pdi_reformer3(df_input):
    pattern_ymd3 = '[0-9]{4}\-[0-9]{2}\-[0-9]{2}'
    str_ymd3 = re.findall(pattern_ymd3, df_input['ENROLL_DATE'].iloc[i])
    if len(str_ymd3) != 0:
        str_ymd3_ = str_ymd3[0].replace('-','')
        df_input.loc[i,'ENROLL_DATE'] = str_ymd3_


# yyyy/mm/dd -> yyyymmdd
# 이 코드 작성 시점까지 yyyy/m/d 등의 패턴은 발견되지 않음.
def pdi_reformer4(df_input):
    pattern_ymd4 = '[0-9]{4}\/[0-9]{2}\/[0-9]{2}'
    str_ymd4 = re.findall(pattern_ymd4, df_input['ENROLL_DATE'].iloc[i])
    if len(str_ymd4) != 0:
        str_ymd4_ = str_ymd4[0].replace('/','')
        df_input.loc[i,'ENROLL_DATE'] = str_ymd4_

In [ ]:
### NEW
### 전처리 함수(엑셀)
def preprocess_xlsx(df_input):
    
    cprt_type = ['img_opentype01', 'img_opentype02', 'img_opentype03', 'img_opentype04']
    ptn_pdi1 = ' [0-9]{2}\:[0-9]{2}\:[0-9]{2}'
    ptn_pdi2 = ' [0-9]{2}\:[0-9]{2}'
    
    # BBS ID
    df_input['BBS_ID'] = None
    # CATEGORY ID
    df_input['CATEGORY_ID'] = None
    # CATEGORY NAME
    df_input['CATEGORY_NAME'] = None
    # BBS NAME
    df_input['BBS_NAME'] = None
    # ORG CODE
    df_input['ORG_CODE'] = None
    # ATC TYPE CODE
    df_input['ATC_TYPE_CODE'] = None
    # CAT CODE 1
    df_input['CAT_CODE1'] = None
    # CAT CODE 2
    df_input['CAT_CODE2'] = None
    
    # ORG URL
    df_input['ORG_URL'] = None
    # DATA GROUP
    df_input['DATA_GROUP'] = None
    
    
     # 저작권
    df_input['RGT_TYPE_CODE'] = ''
    for i in range(len(df_input)):
        if cprt_type[0] in df_input['RGT_TYPE_CODE_'].iloc[i]:
            df_input.loc[i, 'RGT_TYPE_CODE'] = '1'
        elif cprt_type[1] in df_input['RGT_TYPE_CODE_'].iloc[i]:
            df_input.loc[i, 'RGT_TYPE_CODE'] = '2'
        elif cprt_type[2] in df_input['RGT_TYPE_CODE_'].iloc[i]:
            df_input.loc[i, 'RGT_TYPE_CODE'] = '3'
        elif cprt_type[3] in df_input['RGT_TYPE_CODE_'].iloc[i]:
            df_input.loc[i, 'RGT_TYPE_CODE'] = '4'
        else:
            df_input.loc[i, 'RGT_TYPE_CODE'] = '5'
    df_input.drop('RGT_TYPE_CODE_', inplace=True, axis=1)
    

    # 작성일 처리
    # -> pdi_reformer 함수들 사용

### Run (URL method - new) (Xpath)

##### URL 을 이용한 페이지 이동 (Xpath)

##### Run All이면 Xpath, CSS 둘 중에 하나만

In [82]:
### --------------------------------------------------------------------------------------------------------------
print('데이터 수집을 시작합니다.')
### 시작 시각
start_dt = datetime.now().strftime(dt_format)
print(start_dt)
### ---------------------------------------
driver = webdriver.Chrome(options=options)
# 브라우저 창 크기 최대화
driver.maximize_window()
#driver.get(url_input)
url_pg_lst, url_lst, ttl_lst, dt_lst, cntnt_lst, cprt_lst = [], [], [], [], [], []
# 설정시의 파라미터와 실제 수집시의 파라미터가 다를 수 있으므로 예외처리
try:
    for i in range(tot_page):

        # 페이지 클릭보다 주소를 통한 페이지 접근으로 수정...
        url_input = str(url_str_.format(start_page+i))
        driver.get(url_input)
        
        # 랜덤 시간 쉬기
        num_rand = random.uniform(0.01, 0.99)
        print(num_rand)
        time.sleep(num_rand)
        
        ## 예외 처리
        # 마지막 페이지일 때, 수집 페이지 개수가 2개 이상이고 마지막 글 개수가 0이 아닌 상황
        if i == tot_page - 1 and tot_page > 1 and num_a_fin != 0:
            cnt_a = num_a_fin
        # 페이지 개수 1개이고 모으는 글의 개수가 한 페이지에 있어서 글 최대 개수 이하일 때
        if tot_page == 1 and num_a_fin != 0 and num_a_fin <= cnt_a:
            cnt_a = num_a_fin
        # 페이지 개수 0개(start, end 같은 페이지)인데 모으는 글의 개수가 0개 이면 수집 취소
        if tot_page == 0 and num_a_fin == 0:
            print('수집할 데이터가 없습니다.')
        ### 글 클릭하고 데이터 수집
        for j in range(cnt_a):
            # 글 제목 클릭 (마지막 페이지 제외하고 한 페이지 글 개수만큼)
            driver.find_element(By.XPATH, article_attr.format(j + 1)).click()
            # 그림 클릭
            ##driver.find_element(By.XPATH, pic_attr.format(j + 1)).click()
            # 작성일 클릭
            ##driver.find_element(By.XPATH, dt_click_attr.format(j + 1)).click()
            
            ### 데이터 수집
            # ATC_LIST_URL
            # 페이지 URL (글마다 입력 필요)
            url_pg_lst.append(url_input)
        
            # ATC_URL
            # 글 URL
            url_str = str(driver.current_url)
            url_lst.append(url_str)
            # -------------------------------------------------------
            # ATC_TITLE
            # 글 제목
            try:
                try:
                    title = driver.find_element(By.XPATH, ttl_attr1).text
                except:
                    title = driver.find_element(By.XPATH, ttl_attr2).text
            except:
                print('제목이 획득되지 않았습니다.')
                title = ''
            ttl_lst.append(title)
            ### ----------------------------------------------------------------
            # ENROLL_DATE
            # 작성일
            try:
                try:
                    date = driver.find_element(By.XPATH, dt_att1).text
                except:
                    date = driver.find_element(By.XPATH, dt_attr2).text
            except:
                print('작성일이 획득되지 않았습니다.')
                date = ''
            dt_lst.append(date)
            ### ----------------------------------------------------------------
            # ATC_CONTENTS
            # 내용
            try:
                try:
                    cntnt = driver.find_element(By.XPATH, cntnt_attr1).text
                except:
                    cntnt = driver.find_element(By.XPATH, cntnt_attr2).text
            except:
                print('본문 내용이 획득되지 않았습니다.')
                cntnt = ''
            cntnt_lst.append(cntnt)
            ### -----------------------------------------------------------------------------
            # RGT_TYPE_CODE_
            # 저작권 유형
            try:
                try:
                    cprt = driver.find_element(By.XPATH, cprt_attr1).get_attribute('src')
                except:
                    cprt = driver.find_element(By.XPATH, cprt_attr2).get_attribute('src')
            except:
                print('저작권 유형이 획득되지 않았습니다.')
                cprt = ''
            cprt_lst.append(cprt)
            # --------------------------------------------------------------------------------
            # 완료 표시
            print('{} 페이지 {}번째 글 완료'.format((i + start_page), (j + 1)))
            # 랜덤 시간 쉬기
            num_rand = random.uniform(0.01, 0.09)
            print(num_rand)
            time.sleep(num_rand)

            # 뒤로가기 
            ##driver.back()
            # 뒤로가기 대신 url로 게시판 다시 호출(headless를 사용하기 위함)
            url_input = str(url_str_.format(start_page+i))
            driver.get(url_input)
            
            # 랜덤 시간 쉬기
            num_rand = random.uniform(0.71, 1.59)
            print(num_rand)
            time.sleep(num_rand)
            
        if i == tot_page - 1:
            ### 종료 시각
            end_dt = datetime.now().strftime(dt_format)
            print(end_dt)
            ### 종료
            driver.quit()
            
            print('데이터 수집이 완료되었습니다.')
            
except:
    print('파라미터 설정을 다시 확인해 주세요.')
    ### 종료 시각
    end_dt = datetime.now().strftime(dt_format)
    print(end_dt)
    ### 종료
    driver.quit()
    

2023-09-12 16:12:59
1.8954157682453188
2073 페이지 1번째 글 완료
0.019212335066273838
1.097933101228679
2073 페이지 2번째 글 완료
0.016358494319580474
1.3688478310648258
2023-09-12 16:13:10
데이터 수집이 완료되었습니다.


### Run (URL method - new) (CSS)

##### URL 을 이용한 페이지 이동 (CSS)

##### Run All이면 Xpath, CSS 둘 중에 하나만

In [83]:
# ### --------------------------------------------------------------------------------------------------------------
# print('데이터 수집을 시작합니다.')
# ### 시작 시각
# start_dt = datetime.now().strftime(dt_format)
# print(start_dt)
# ### ---------------------------------------
# driver = webdriver.Chrome(options=options)
# driver.maximize_window()
# #driver.get(url_input)
# url_pg_lst, url_lst, ttl_lst, dt_lst, cntnt_lst, cprt_lst = [], [], [], [], [], []
# # 설정시의 파라미터와 실제 수집시의 파라미터가 다를 수 있으므로 예외처리
# try:
#     for i in range(tot_page):

#         # 페이지 클릭보다 주소를 통한 페이지 접근으로 수정...
#         url_input = str(url_str_.format(start_page+i))
#         driver.get(url_input)
        
#         # 랜덤 시간 쉬기
#         num_rand = random.uniform(0.01, 0.99)
#         print(num_rand)
#         time.sleep(num_rand)
        
#         ## 예외 처리
#         # 마지막 페이지일 때, 수집 페이지 개수가 2개 이상이고 마지막 글 개수가 0이 아닌 상황
#         if i == tot_page - 1 and tot_page > 1 and num_a_fin != 0:
#             cnt_a = num_a_fin
#         # 페이지 개수 1개이고 모으는 글의 개수가 한 페이지에 있어서 글 최대 개수 이하일 때
#         if tot_page == 1 and num_a_fin != 0 and num_a_fin <= cnt_a:
#             cnt_a = num_a_fin
#         # 페이지 개수 0개(start, end 같은 페이지)인데 모으는 글의 개수가 0개 이면 수집 취소
#         if tot_page == 0 and num_a_fin == 0:
#             print('수집할 데이터가 없습니다.')
#         ### 글 클릭하고 데이터 수집
#         for j in range(cnt_a):
#             # 글 제목 클릭 (마지막 페이지 제외하고 10개씩)
#             driver.find_element(By.CSS_SELECTOR, article_attr.format(j + 1)).click()
#             # 그림 클릭
#             ##driver.find_element(By.CSS_SELECTOR, pic_attr.format(j + 1)).click()
#             # 작성일 클릭
#             ##driver.find_element(By.CSS_SELECTOR, dt_click_attr.format(j + 1)).click()
            
#             ### 데이터 수집
#             # ATC_LIST_URL
#             # 페이지 URL (글마다 입력 필요)
#             url_pg_lst.append(url_input)
        
#             # ATC_URL
#             # 글 URL
#             url_str = str(driver.current_url)
#             url_lst.append(url_str)
#             # -------------------------------------------------------
#             # ATC_TITLE
#             # 글 제목
#             try:
#                 try:
#                     title = driver.find_element(By.CSS_SELECTOR, ttl_attr1).text
#                 except:
#                     title = driver.find_element(By.CSS_SELECTOR, ttl_attr2).text
#             except:
#                 print('제목이 획득되지 않았습니다.')
#                 title = ''
#             ttl_lst.append(title)
#             ### ----------------------------------------------------------------
#             # ENROLL_DATE
#             # 작성일
#             try:
#                 try:
#                     date = driver.find_element(By.CSS_SELECTOR, dt_att1).text
#                 except:
#                     date = driver.find_element(By.CSS_SELECTOR, dt_attr2).text
#             except:
#                 print('작성일이 획득되지 않았습니다.')
#                 date = ''
#             dt_lst.append(date)
#             ### ----------------------------------------------------------------
#             # ATC_CONTENTS
#             # 내용
#             try:
#                 try:
#                     cntnt = driver.find_element(By.CSS_SELECTOR, cntnt_attr1).text
#                 except:
#                     cntnt = driver.find_element(By.CSS_SELECTOR, cntnt_attr2).text
#             except:
#                 print('본문 내용이 획득되지 않았습니다.')
#                 cntnt = ''
#             cntnt_lst.append(cntnt)
#             ### -----------------------------------------------------------------------------
#             # RGT_TYPE_CODE_
#             # 저작권 유형
#             try:
#                 try:
#                     cprt = driver.find_element(By.CSS_SELECTOR, cprt_attr1).get_attribute('src')
#                 except:
#                     cprt = driver.find_element(By.CSS_SELECTOR, cprt_attr2).get_attribute('src')
#             except:
#                 print('저작권 유형이 획득되지 않았습니다.')
#                 cprt = ''
#             cprt_lst.append(cprt)
#             # --------------------------------------------------------------------------------
#             # 완료 표시
#             print('{} 페이지 {}번째 글 완료'.format((i + start_page), (j + 1)))
#             # 랜덤 시간 쉬기
#             num_rand = random.uniform(0.01, 0.09)
#             print(num_rand)
#             time.sleep(num_rand)

#             # 뒤로가기 
#             ##driver.back()
#             # 뒤로가기 대신 url로 게시판 다시 호출(headless를 사용하기 위함)
#             url_input = str(url_str_.format(start_page+i))
#             driver.get(url_input)
            
#             # 랜덤 시간 쉬기
#             num_rand = random.uniform(0.71, 1.59)
#             print(num_rand)
#             time.sleep(num_rand)
            
#         if i == tot_page - 1:
#             ### 종료 시각
#             end_dt = datetime.now().strftime(dt_format)
#             print(end_dt)
#             ### 종료
#             driver.quit()
            
#             print('데이터 수집이 완료되었습니다.')
            
# except:
#     print('파라미터 설정을 다시 확인해 주세요.')
#     ### 종료 시각
#     end_dt = datetime.now().strftime(dt_format)
#     print(end_dt)
#     ### 종료
#     driver.quit()
    

### Run (URL method - old) (Xpath)

In [ ]:
# ### Browsing 여부 ### ### ### ### ### ### ### ### ### ### ### 
# options = Options()
# # 이미지 로딩 OFF (속도 개선)
# options.add_argument('--blink-settings=imagesEnabled=false')
# #options.add_argument("--window-size=1600,1000")
# options.add_argument('-headless')
# ### --------------------------------------------------------------------------------------------------------------
# ### 시작 시각
# start_dt = datetime.now().strftime(dt_format)
# print(start_dt)
# ### ---------------------------------------
# driver = webdriver.Chrome(options=options)
# driver.maximize_window()
# #driver.get(url_input)
 
# url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2 = [], [], [], [], [], [], [], []
# # 설정시의 파라미터와 실제 수집시의 파라미터가 다를 수 있으므로 예외처리
# try:
#     for i in range(tot_page):

#         # 페이지 클릭보다 주소를 통한 페이지 접근으로 수정...
#         url_input = str(url_str_.format(start_page+i))
#         driver.get(url_input)
        
#         # 사람처럼 보이기 # 반복 횟수 많아질 때 sleep 길게해 줄 필요있음(10,000개 까진 필요없는 것 같음)
#         num_tst = random.uniform(0.01, 0.99)
#         print(num_tst)
#         time.sleep(num_tst)
#         ### 글 클릭하고 데이터 수집
#         ## 예외 처리
#         # 마지막 페이지일 때, 수집 페이지 개수가 2개 이상이고 마지막 글 개수가 0이 아닌 상황
#         if i == tot_page - 1 and tot_page > 1 and num_a_fin != 0:
#             cnt_a = num_a_fin
#         # 페이지 개수 1개이고 모으는 글의 개수가 한 페이지에 있어서 글 최대 개수 이하일 때
#         if tot_page == 1 and num_a_fin != 0 and num_a_fin <= cnt_a:
#             cnt_a = num_a_fin
#         # 페이지 개수 0개(start, end 같은 페이지)인데 모으는 글의 개수가 0개 이면 수집 취소
#         if tot_page == 0 and num_a_fin == 0:
#             print('수집할 데이터가 없습니다.')
#             df_out = pd.DataFrame(np.column_stack([url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2]),
#                     columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM', 'CP_TYPE_NM_','tm_stamp1', 'tm_stamp2'])
#         for j in range(cnt_a):
#             # 글 클릭 (마지막 페이지 제외하고 10개씩)
#             driver.find_element(By.XPATH, article_attr.format(j + 1)).click()
#             #driver.find_element(By.XPATH, pic_attr.format(j + 1)).click()
#             #driver.find_element(By.XPATH, dt_click_attr.format(j + 1)).click()
#             ### 데이터 수집
#             # URL
#             url_str = str(driver.current_url)
#             url_lst.append(url_str)
#             # -------------------------------------------------------
#             # time stamp 1
#             stmp1 = datetime.now().strftime(dt_format)
#             tm_stmp1.append(stmp1)
#             # -----------------------------------------------------------------
#             # 글 제목
#             title = driver.find_element(By.XPATH, ttl_attr).text
#             #print(title)
#             if len(title) == 0:
#                 ttl_lst.append('')
#             else:
#                 ttl_lst.append(title)
#             ### ----------------------------------------------------------------
#             # 작성일
#             date = driver.find_element(By.XPATH, dt_attr).text
#             if len(date) == 0:
#                 dt_lst.append('')
#             else:
#                 dt_lst.append(date)
#             ### ----------------------------------------------------------------
#             # 내용
#             try:
#                 try:
#                     cntnt1 = driver.find_element(By.XPATH, cntnt_attr1).text
#                     cntnt = cntnt1
#                 except:
#                     cntnt2 = driver.find_element(By.XPATH, cntnt_attr2).text
#                     cntnt = cntnt2
#             except:
#                 print('본문 내용이 획득되지 않았습니다.')
#                 cntnt = ''
#             cntnt_lst.append(cntnt)
#             ### ----------------------------------------------------------------
#             # 파일명
#             file_nm = driver.find_elements(By.XPATH, fnm_attr)
#             if len(file_nm) == 0:
#                 print('파일 다운로드 없음')
#                 fnm_lst.append('')
#             else:
#                 file__ = []
#                 for file_ in file_nm:
#                     ##print(file_.text)
#                     file__.append(file_.text)
#                 fnm_lst.append(file__[0])
#             ### -----------------------------------------------------------------------------
#             # 저작권 유형
#             try:
#                 try:
#                     cprt1 = driver.find_element(By.XPATH, cprt_attr1).get_attribute('src')
#                     cprt = cprt1
#                 except:
#                     cprt2 = driver.find_element(By.XPATH, cprt_attr2).get_attribute('src')
#                     cprt = cprt2
#             except:
#                 print('저작권 유형이 획득되지 않았습니다.')
#                 #print('cprt1 except : {}'.format(cprt1))
#                 #print('cprt2 except : {}'.format(cprt2))
#                 cprt = ''
#             cprt_lst.append(cprt)
#             # --------------------------------------------------------------------------------
#             # 로깅
#             print('{} 페이지 {}번째 글 완료'.format((i + start_page), (j + 1)))
#             # 사람처럼 보이기 # 반복 횟수 많아질 때 sleep 길게해 줄 필요있음(10,000개 까진 필요없는 것 같음)
#             num_tst = random.uniform(0.01, 0.09)
#             print(num_tst)
#             time.sleep(num_tst)

#             # time stamp 2
#             stmp2 = datetime.now().strftime(dt_format)
#             tm_stmp2.append(stmp2)

#             # 뒤로가기 # 뒤로가기 대신 url로 게시판 다시 호출(headless를 사용하기 위함)
#             #driver.back()
#             url_input = str(url_str_.format(start_page+i))
#             driver.get(url_input)
            
#             # 사람처럼 보이기 # 반복 횟수 많아질 때 sleep 길게해 줄 필요있음(10,000개 까진 필요없는 것 같음)
#             num_tst = random.uniform(0.71, 1.59)
#             print(num_tst)
#             time.sleep(num_tst)
            
#         if i == tot_page - 1:
#             ### 종료 시각
#             end_dt = datetime.now().strftime(dt_format)
#             print(end_dt)
#             ### 종료
#             driver.quit()
            
#             print('데이터 수집이 완료되었습니다.')
            
#             ## df
# #             df_out = pd.DataFrame(np.column_stack([url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2]),
# #                 columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM','CP_TYPE_NM_', 'tm_stamp1','tm_stamp2'])
# except:
#     print('파라미터 설정을 다시 확인해 주세요.')
#     ### 종료 시각
#     end_dt = datetime.now().strftime(dt_format)
#     print(end_dt)
#     ### 종료
#     driver.quit()
#     ### df
# #     df_out = pd.DataFrame(np.column_stack([url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2]),
# #             columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM','CP_TYPE_NM_', 'tm_stamp1','tm_stamp2'])

### Run (URL method - old) (CSS)

In [ ]:
# ### Browsing 여부 ### ### ### ### ### ### ### ### ### ### ### 
# options = Options()
# # 이미지 로딩 OFF (속도 개선)
# options.add_argument('--blink-settings=imagesEnabled=false')
# #options.add_argument("--window-size=1600,1000")
# #options.add_argument('headless')
# ### --------------------------------------------------------------------------------------------------------------
# ### 시작 시각
# start_dt = datetime.now().strftime(dt_format)
# print(start_dt)
# ### ---------------------------------------
# driver = webdriver.Chrome(options=options)
# driver.maximize_window()
# #driver.get(url_input)
 
# url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2 = [], [], [], [], [], [], [], []
# # 설정시의 파라미터와 실제 수집시의 파라미터가 다를 수 있으므로 예외처리
# try:
#     for i in range(tot_page):

#         # 페이지 클릭보다 주소를 통한 페이지 접근으로 수정...
#         url_input = str(url_str_.format(start_page+i))
#         driver.get(url_input)
        
#         # 사람처럼 보이기 # 반복 횟수 많아질 때 sleep 길게해 줄 필요있음(10,000개 까진 필요없는 것 같음)
#         num_tst = random.uniform(1.01, 1.99)
#         print(num_tst)
#         time.sleep(num_tst)

#         ### 글 클릭하고 데이터 수집
#         ## 예외 처리
#         # 마지막 페이지일 때, 수집 페이지 개수가 2개 이상이고 마지막 글 개수가 0이 아닌 상황
#         if i == tot_page - 1 and tot_page > 1 and num_a_fin != 0:
#             cnt_a = num_a_fin
#         # 페이지 개수 1개이고 모으는 글의 개수가 한 페이지에 있어서 글 최대 개수 이하일 때
#         if tot_page == 1 and num_a_fin != 0 and num_a_fin <= cnt_a:
#             cnt_a = num_a_fin
#         # 페이지 개수 0개(start, end 같은 페이지)인데 모으는 글의 개수가 0개 이면 수집 취소
#         if tot_page == 0 and num_a_fin == 0:
#             print('수집할 데이터가 없습니다.')
#             df_out = pd.DataFrame(np.column_stack([url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2]),
#                     columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM', 'CP_TYPE_NM_','tm_stamp1', 'tm_stamp2'])
#         for j in range(cnt_a):
#             # 글 클릭 (마지막 페이지 제외하고 10개씩)
#             driver.find_element(By.CSS_SELECTOR, article_attr.format(j + 1)).click()
#             #driver.find_element(By.CSS_SELECTOR, pic_attr.format(j + 1)).click()
#             #driver.find_element(By.CSS_SELECTOR, dt_click_attr.format(j + 1)).click()
#             ### 데이터 수집
#             # URL
#             url_str = str(driver.current_url)
#             url_lst.append(url_str)
#             # -------------------------------------------------------
#             # time stamp 1
#             stmp1 = datetime.now().strftime(dt_format)
#             tm_stmp1.append(stmp1)
#             # -----------------------------------------------------------------
#             # 글 제목
#             title = driver.find_element(By.CSS_SELECTOR, ttl_attr).text
#             #print(title)
#             if len(title) == 0:
#                 ttl_lst.append('')
#             else:
#                 ttl_lst.append(title)
#             ### ----------------------------------------------------------------
#             # 작성일
#             date = driver.find_element(By.CSS_SELECTOR, dt_attr).text
#             if len(date) == 0:
#                 dt_lst.append('')
#             else:
#                 dt_lst.append(date)
#             ### ----------------------------------------------------------------
#             # 내용
#             try:
#                 try:
#                     cntnt1 = driver.find_element(By.CSS_SELECTOR, cntnt_attr1).text
#                     cntnt = cntnt1
#                 except:
#                     cntnt2 = driver.find_element(By.CSS_SELECTOR, cntnt_attr2).text
#                     cntnt = cntnt2
#             except:
#                 print('본문 내용이 획득되지 않았습니다.')
#                 cntnt = ''
#             cntnt_lst.append(cntnt)
#             ### ----------------------------------------------------------------
#             # 파일명
#             file_nm = driver.find_elements(By.CSS_SELECTOR, fnm_attr)
#             if len(file_nm) == 0:
#                 print('파일 다운로드 없음')
#                 fnm_lst.append('')
#             else:
#                 file__ = []
#                 for file_ in file_nm:
#                     ##print(file_.text)
#                     file__.append(file_.text)
#                 fnm_lst.append(file__[0])
#             ### -----------------------------------------------------------------------------
#             # 저작권 유형
#             try:
#                 try:
#                     cprt1 = driver.find_element(By.CSS_SELECTOR, cprt_attr1).get_attribute('src')
#                     cprt = cprt1
#                 except:
#                     cprt2 = driver.find_element(By.CSS_SELECTOR, cprt_attr2).get_attribute('src')
#                     cprt = cprt2
#             except:
#                 print('저작권 유형이 획득되지 않았습니다.')
#                 #print('cprt1 except : {}'.format(cprt1))
#                 #print('cprt2 except : {}'.format(cprt2))
#                 cprt = ''
#             cprt_lst.append(cprt)
#             # --------------------------------------------------------------------------------
#             # 로깅
#             print('{} 페이지 {}번째 글 완료'.format((i + start_page), (j + 1)))
#             # 사람처럼 보이기 # 반복 횟수 많아질 때 sleep 길게해 줄 필요있음(10,000개 까진 필요없는 것 같음)
#             num_tst = random.uniform(0.01, 0.09)
#             print(num_tst)
#             time.sleep(num_tst)

#             # time stamp 2
#             stmp2 = datetime.now().strftime(dt_format)
#             tm_stmp2.append(stmp2)

#             # 뒤로가기
#             driver.back()
#             # 사람처럼 보이기 # 반복 횟수 많아질 때 sleep 길게해 줄 필요있음(10,000개 까진 필요없는 것 같음)
#             num_tst = random.uniform(1.01, 2.09)
#             print(num_tst)
#             time.sleep(num_tst)
            
#         if i == tot_page - 1:
#             ### 종료 시각
#             end_dt = datetime.now().strftime(dt_format)
#             print(end_dt)
#             ### 종료
#             driver.quit()
            
#             print('데이터 수집이 완료되었습니다.')
            
#             ## df
# #             df_out = pd.DataFrame(np.column_stack([url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2]),
# #                 columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM','CP_TYPE_NM_', 'tm_stamp1','tm_stamp2'])
# except:
#     print('파라미터 설정을 다시 확인해 주세요.')
#     ### 종료 시각
#     end_dt = datetime.now().strftime(dt_format)
#     print(end_dt)
#     ### 종료
#     driver.quit()
#     ### df
# #     df_out = pd.DataFrame(np.column_stack([url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2]),
# #             columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM','CP_TYPE_NM_', 'tm_stamp1','tm_stamp2'])

### Run (Click-Button method) (Xpath)

##### 버튼 클릭을 통한 페이지 이동 (URL 주소가 불규칙하게 변형될 때 사용 권장)

In [ ]:
# ### --------------------------------------------------------------------------------------------------------------
# ### 시작 시각
# start_dt = datetime.now().strftime(dt_format)
# print(start_dt)
# ### ---------------------------------------
# driver = webdriver.Chrome(options=options)
# #driver = webdriver.Firefox(options=options)
# driver.maximize_window()
# #driver.execute_script("document.body.style.MozTransform = 'scale(0.1)'")
# driver.get(url_input)
# time.sleep(5)

# url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2 = [], [], [], [], [], [], [], []
# # 설정시의 파라미터와 실제 수집시의 파라미터가 다를 수 있으므로 예외처리
# try:
#     for i in range(tot_page):

#         ### 페이지 클릭
#         if cnt_cp == 0:
#             cnt_cp = page_cnt
     
#         # 해당 페이지 클릭 # 현재 페이지가 굵게 표시되고 클릭이 안 되는 게시판도 있으므로 기본적으로는 주석처리함.
#         #driver.find_element(By.XPATH, cpage_attr.format(cnt_cp)).click()
        
#         #driver.find_element(By.XPATH, cpage_attr).click()
#         # 사람처럼 보이기 # 반복 횟수 많아질 때 sleep 길게해 줄 필요있음(10,000개 까진 필요없는 것 같음)
#         num_tst = random.uniform(0.01, 0.09)
#         print(num_tst)
#         time.sleep(num_tst)

#         ### 글 클릭하고 데이터 수집
#         ## 예외 처리
#         # 마지막 페이지일 때, 수집 페이지 개수가 2개 이상이고 마지막 글 개수가 0이 아닌 상황
#         if i == tot_page - 1 and tot_page > 1 and num_a_fin != 0:
#             cnt_a = num_a_fin
#         # 페이지 개수 1개이고 모으는 글의 개수가 한 페이지에 있어서 글 최대 개수 이하일 때
#         if tot_page == 1 and num_a_fin != 0 and num_a_fin <= cnt_a:
#             cnt_a = num_a_fin
#         # 페이지 개수 0개(start, end 같은 페이지)인데 모으는 글의 개수가 0개 이면 수집 취소
#         if tot_page == 0 and num_a_fin == 0:
#             print('수집할 데이터가 없습니다.')
#             df_out = pd.DataFrame(np.column_stack([url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2]),
#                     columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM', 'CP_TYPE_NM_','tm_stamp1', 'tm_stamp2'])
#         for j in range(cnt_a):
#             # 글 클릭 (마지막 페이지 제외하고 10개씩)
          
#             #print(article_attr.format(j + 1))
#             #prev_height = driver.execute_script('return document.body.scrollHeight')
#             #print('prev height : {}'.format(prev_height))
#             # 게시판 사이즈가 클 경우 스크롤 다운 등의 동작 추가도 필요(항시 해당되는 것은 아니므로 주석처리됨)
#             #driver.execute_script('window.scrollTo(0,340)')
           
#             driver.find_element(By.XPATH, article_attr1.format(j + 1)).click()
#             #driver.find_element(By.XPATH, r'//*[@id="boardItem"]/a[{}]'.format(j+1)).click()
#             #driver.find_element(By.XPATH, '#boardItem > a:nth-child({})'.format(j+1)).click()
#             #'//*[@id="boardItem"]/a[1]'
#             # 글 클릭에 드는 시간에 비해 게시글 로딩 시간이 길 경우 sleep 주는 것이 필요.
#             #time.sleep(5)
#             #time.sleep(10)
#             # 글 제목이 없을 경우, 사진 혹은 작성일 클릭을 권장
#             # driver.find_element(By.XPATH, pic_attr.format(j + 1)).click()
#             #driver.find_element(By.XPATH, dt_click_attr.format(j + 1)).click()
#             ### 데이터 수집
#             # URL
#             url_str = str(driver.current_url)
#             url_lst.append(url_str)
#             # -------------------------------------------------------
#             # time stamp 1
#             stmp1 = datetime.now().strftime(dt_format)
#             tm_stmp1.append(stmp1)
#             # -----------------------------------------------------------------
#             # 글 제목
#             try:
#                 try:
#                     title = driver.find_element(By.XPATH, ttl_attr1).text
#                 except:
#                     title = driver.find_element(By.XPATH, ttl_attr2).text
#             except:
#                 print('제목이 획득되지 않았습니다.')
#                 title = ''
#             ttl_lst.append(title)
#             # 확인용 제목 출력
#             #print('title : {}'.format(title))
#             # -----------------------------------------------------------------
#             # 작성일
#             try:
#                 try:
#                     date = driver.find_element(By.XPATH, dt_attr1).text
#                 except:
#                     date = driver.find_element(By.XPATH, dt_attr2).text
#             except:
#                 print('작성일이 획득되지 않았습니다.')
#                 date = ''
#             dt_lst.append(date)
#             # -----------------------------------------------------------------
#             # 내용
#             try:
#                 try:
#                     cntnt = driver.find_element(By.XPATH, cntnt_attr1).text
#                 except:
#                     cntnt = driver.find_element(By.XPATH, cntnt_attr2).text
#             except:
#                 print('본문 내용이 획득되지 않았습니다.')
#                 cntnt = ''
#             cntnt_lst.append(cntnt)
#             ### ----------------------------------------------------------------
#             file_nm = driver.find_elements(By.XPATH, fnm_attr)
#             if len(file_nm) == 0:
#                 print('파일 다운로드 없음')
#                 fnm_lst.append('')
#             else:
#                 file__ = []
#                 for file_ in file_nm:
#                     ##print(file_.text)
#                     file__.append(file_.text)
#                 fnm_lst.append(file__[0])
#             # 파일 다운로드 태그가 구현이 안된 게시판의 경우 위 파일명 획득 코드를 주석처리 하고
#             # 밑의 '' 빈 스트링 값을 추가하는 코드의 주석을 풀기 바람
#             #fnm_lst.append('')
#             ### -----------------------------------------------------------------------------
#             # 저작권 유형
#             try:
#                 try:
#                     cprt = driver.find_element(By.XPATH, cprt_attr1).get_attribute('src')
#                 except:
#                     cprt = driver.find_element(By.XPATH, cprt_attr2).get_attribute('src')
#             except:
#                 print('저작권 유형이 획득되지 않았습니다.')
#                 cprt = ''
#             cprt_lst.append(cprt)
#             # --------------------------------------------------------------------------------
#             # 로깅
#             print('{} 페이지 {}번째 글 완료'.format((i + start_page), (j + 1)))
#             # 사람처럼 보이기 # 반복 횟수 많아질 때 sleep 길게해 줄 필요있음(10,000개 까진 필요없는 것 같음)
#             num_tst = random.uniform(0.01, 0.09)
#             print(num_tst)
#             time.sleep(num_tst)

#             # time stamp 2
#             stmp2 = datetime.now().strftime(dt_format)
#             tm_stmp2.append(stmp2)

#             time.sleep(5)
            
#             # 뒤로가기
#             driver.back()
                
#             time.sleep(5)

#             # 사람처럼 보이기 # 반복 횟수 많아질 때 sleep 길게해 줄 필요있음(10,000개 까진 필요없는 것 같음)
#             num_tst = random.uniform(1.01, 2.09)
#             print(num_tst)
#             time.sleep(num_tst)

#             print('cnt cp : {}'.format(cnt_cp))

#         # 하단 페이지 개수에서 최대 번호 페이지일 때
#         if cnt_cp == page_cnt:
#             # > 버튼(다음 페이지 이동 버튼) 클릭
#             driver.find_element(By.XPATH, nxt_btn_attr).click()
#             # 하단 페이지 개수에서 최대 페이지 번호 마다 카운트 초기화
#             cnt_cp = 1
#         else:
#             # 카운트 증가
#             cnt_cp += 1
            
#         # 마지막 페이지이고 루프가 거의 끝나갈 때
#         if i == tot_page - 1:
#             ### 종료 시각
#             end_dt = datetime.now().strftime(dt_format)
#             print(end_dt)
#             ### 종료
#             driver.quit()

#             print('데이터 수집이 완료되었습니다.')

#             ## df
#             df_out = pd.DataFrame(np.column_stack([url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2]),
#                     columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM', 'CP_TYPE_NM_','tm_stamp1', 'tm_stamp2'])
# except:
#     print('파라미터 설정을 다시 확인해 주세요.')
#     ### 종료 시각
#     end_dt = datetime.now().strftime(dt_format)
#     print(end_dt)
#     ### 종료
#     driver.quit()
#     ### df
#     df_out = pd.DataFrame(np.column_stack([url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2]),
#             columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM', 'CP_TYPE_NM_', 'tm_stamp1','tm_stamp2'])

### Run (Click-Button method) (CSS)

##### 버튼 클릭을 통한 페이지 이동 (URL 주소가 불규칙하게 변형될 때 사용 권장)

In [ ]:
# ### --------------------------------------------------------------------------------------------------------------
# ### 시작 시각
# start_dt = datetime.now().strftime(dt_format)
# print(start_dt)
# ### ---------------------------------------
# driver = webdriver.Chrome(options=options)
# #driver = webdriver.Firefox(options=options)
# driver.maximize_window()
# #driver.execute_script("document.body.style.MozTransform = 'scale(0.1)'")
# driver.get(url_input)
# time.sleep(5)

# url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2 = [], [], [], [], [], [], [], []
# # 설정시의 파라미터와 실제 수집시의 파라미터가 다를 수 있으므로 예외처리
# try:
#     for i in range(tot_page):

#         ### 페이지 클릭
#         if cnt_cp == 0:
#             cnt_cp = page_cnt
     
#         # 해당 페이지 클릭 # 현재 페이지가 굵게 표시되고 클릭이 안 되는 게시판도 있으므로 기본적으로는 주석처리함.
#         #driver.find_element(By.CSS_SELECTOR, cpage_attr.format(cnt_cp)).click()
        
#         #driver.find_element(By.CSS_SELECTOR, cpage_attr).click()
#         # 사람처럼 보이기 # 반복 횟수 많아질 때 sleep 길게해 줄 필요있음(10,000개 까진 필요없는 것 같음)
#         num_tst = random.uniform(0.01, 0.09)
#         print(num_tst)
#         time.sleep(num_tst)

#         ### 글 클릭하고 데이터 수집
#         ## 예외 처리
#         # 마지막 페이지일 때, 수집 페이지 개수가 2개 이상이고 마지막 글 개수가 0이 아닌 상황
#         if i == tot_page - 1 and tot_page > 1 and num_a_fin != 0:
#             cnt_a = num_a_fin
#         # 페이지 개수 1개이고 모으는 글의 개수가 한 페이지에 있어서 글 최대 개수 이하일 때
#         if tot_page == 1 and num_a_fin != 0 and num_a_fin <= cnt_a:
#             cnt_a = num_a_fin
#         # 페이지 개수 0개(start, end 같은 페이지)인데 모으는 글의 개수가 0개 이면 수집 취소
#         if tot_page == 0 and num_a_fin == 0:
#             print('수집할 데이터가 없습니다.')
#             df_out = pd.DataFrame(np.column_stack([url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2]),
#                     columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM', 'CP_TYPE_NM_','tm_stamp1', 'tm_stamp2'])
#         for j in range(cnt_a):
#             # 글 클릭 (마지막 페이지 제외하고 10개씩)
          
#             #print(article_attr.format(j + 1))
#             #prev_height = driver.execute_script('return document.body.scrollHeight')
#             #print('prev height : {}'.format(prev_height))
#             # 게시판 사이즈가 클 경우 스크롤 다운 등의 동작 추가도 필요(항시 해당되는 것은 아니므로 주석처리됨)
#             #driver.execute_script('window.scrollTo(0,340)')
           
#             driver.find_element(By.CSS_SELECTOR, article_attr1.format(j + 1)).click()
#             #driver.find_element(By.XPATH, r'//*[@id="boardItem"]/a[{}]'.format(j+1)).click()
#             #driver.find_element(By.CSS_SELECTOR, '#boardItem > a:nth-child({})'.format(j+1)).click()
#             #'//*[@id="boardItem"]/a[1]'
#             # 글 클릭에 드는 시간에 비해 게시글 로딩 시간이 길 경우 sleep 주는 것이 필요.
#             #time.sleep(5)
#             #time.sleep(10)
#             # 글 제목이 없을 경우, 사진 혹은 작성일 클릭을 권장
#             # driver.find_element(By.CSS_SELECTOR, pic_attr.format(j + 1)).click()
#             #driver.find_element(By.CSS_SELECTOR, dt_click_attr.format(j + 1)).click()
#             ### 데이터 수집
#             # URL
#             url_str = str(driver.current_url)
#             url_lst.append(url_str)
#             # -------------------------------------------------------
#             # time stamp 1
#             stmp1 = datetime.now().strftime(dt_format)
#             tm_stmp1.append(stmp1)
#             # -----------------------------------------------------------------
#             # 글 제목
#             try:
#                 try:
#                     title = driver.find_element(By.CSS_SELECTOR, ttl_attr1).text
#                 except:
#                     title = driver.find_element(By.CSS_SELECTOR, ttl_attr2).text
#             except:
#                 print('제목이 획득되지 않았습니다.')
#                 title = ''
#             ttl_lst.append(title)
#             # 확인용 제목 출력
#             #print('title : {}'.format(title))
#             # -----------------------------------------------------------------
#             # 작성일
#             try:
#                 try:
#                     date = driver.find_element(By.CSS_SELECTOR, dt_attr1).text
#                 except:
#                     date = driver.find_element(By.CSS_SELECTOR, dt_attr2).text
#             except:
#                 print('작성일이 획득되지 않았습니다.')
#                 date = ''
#             dt_lst.append(date)
#             # -----------------------------------------------------------------
#             # 내용
#             try:
#                 try:
#                     cntnt = driver.find_element(By.CSS_SELECTOR, cntnt_attr1).text
#                 except:
#                     cntnt = driver.find_element(By.CSS_SELECTOR, cntnt_attr2).text
#             except:
#                 print('본문 내용이 획득되지 않았습니다.')
#                 cntnt = ''
#             cntnt_lst.append(cntnt)
#             ### ----------------------------------------------------------------
#             file_nm = driver.find_elements(By.CSS_SELECTOR, fnm_attr)
#             if len(file_nm) == 0:
#                 print('파일 다운로드 없음')
#                 fnm_lst.append('')
#             else:
#                 file__ = []
#                 for file_ in file_nm:
#                     ##print(file_.text)
#                     file__.append(file_.text)
#                 fnm_lst.append(file__[0])
#             # 파일 다운로드 태그가 구현이 안된 게시판의 경우 위 파일명 획득 코드를 주석처리 하고
#             # 밑의 '' 빈 스트링 값을 추가하는 코드의 주석을 풀기 바람
#             #fnm_lst.append('')
#             ### -----------------------------------------------------------------------------
#             # 저작권 유형
#             try:
#                 try:
#                     cprt = driver.find_element(By.CSS_SELECTOR, cprt_attr1).get_attribute('src')
#                 except:
#                     cprt = driver.find_element(By.CSS_SELECTOR, cprt_attr2).get_attribute('src')
#             except:
#                 print('저작권 유형이 획득되지 않았습니다.')
#                 cprt = ''
#             cprt_lst.append(cprt)
#             # --------------------------------------------------------------------------------
#             # 로깅
#             print('{} 페이지 {}번째 글 완료'.format((i + start_page), (j + 1)))
#             # 사람처럼 보이기 # 반복 횟수 많아질 때 sleep 길게해 줄 필요있음(10,000개 까진 필요없는 것 같음)
#             num_tst = random.uniform(0.01, 0.09)
#             print(num_tst)
#             time.sleep(num_tst)

#             # time stamp 2
#             stmp2 = datetime.now().strftime(dt_format)
#             tm_stmp2.append(stmp2)

#             time.sleep(5)
            
#             # 뒤로가기
#             driver.back()
                
#             time.sleep(5)

#             # 사람처럼 보이기 # 반복 횟수 많아질 때 sleep 길게해 줄 필요있음(10,000개 까진 필요없는 것 같음)
#             num_tst = random.uniform(1.01, 2.09)
#             print(num_tst)
#             time.sleep(num_tst)

#             print('cnt cp : {}'.format(cnt_cp))

#         # 하단 페이지 개수에서 최대 번호 페이지일 때
#         if cnt_cp == page_cnt:
#             # > 버튼(다음 페이지 이동 버튼) 클릭
#             driver.find_element(By.CSS_SELECTOR, nxt_btn_attr).click()
#             # 하단 페이지 개수에서 최대 페이지 번호 마다 카운트 초기화
#             cnt_cp = 1
#         else:
#             # 카운트 증가
#             cnt_cp += 1
            
#         # 마지막 페이지이고 루프가 거의 끝나갈 때
#         if i == tot_page - 1:
#             ### 종료 시각
#             end_dt = datetime.now().strftime(dt_format)
#             print(end_dt)
#             ### 종료
#             driver.quit()

#             print('데이터 수집이 완료되었습니다.')

#             ## df
#             df_out = pd.DataFrame(np.column_stack([url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2]),
#                     columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM', 'CP_TYPE_NM_','tm_stamp1', 'tm_stamp2'])
# except:
#     print('파라미터 설정을 다시 확인해 주세요.')
#     ### 종료 시각
#     end_dt = datetime.now().strftime(dt_format)
#     print(end_dt)
#     ### 종료
#     driver.quit()
#     ### df
#     df_out = pd.DataFrame(np.column_stack([url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2]),
#             columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM', 'CP_TYPE_NM_', 'tm_stamp1','tm_stamp2'])

In [ ]:
# ### OLD
# len_lst_old = [len(url_lst), len(ttl_lst), len(dt_lst), len(cntnt_lst), len(fnm_lst), len(cprt_lst), len(tm_stmp1), len(tm_stmp2)]
# print(len_lst_old)

In [ ]:
# ### OLD
# if len(url_lst) == len(ttl_lst) == len(dt_lst) == len(cntnt_lst) == len(fnm_lst) == len(cprt_lst) == len(tm_stmp1) == len(tm_stmp2):
#     df_out = pd.DataFrame(np.column_stack([url_lst, ttl_lst, dt_lst, cntnt_lst, fnm_lst, cprt_lst, tm_stmp1, tm_stmp2]),
#                       columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM','CP_TYPE_NM_', 'tm_stamp1','tm_stamp2'])
# else:
#     num_limit = min(len_lst_old)
#     df_out = pd.DataFrame(np.column_stack([url_lst[:num_limit], ttl_lst[:num_limit], dt_lst[:num_limit], cntnt_lst[:num_limit], fnm_lst[:num_limit], cprt_lst[:num_limit], tm_stmp1[:num_limit], tm_stmp2[:num_limit]]),
#                       columns=['URL_ADDR', 'TTL', 'DT_INFO', 'CN', 'FILE_NM','CP_TYPE_NM_', 'tm_stamp1','tm_stamp2'])

### Preprocessing (old)

In [ ]:
# ### OLD
# preprocess(df_out,bbs_sn_num)

In [ ]:
# ### OLD
# folder_nm_old = 'CSV'
# dir_main = os.getcwd()
# os.makedirs(folder_nm_old, exist_ok=True)
# save_dir_old = os.path.join(dir_main,folder_nm_old)

In [ ]:
# ### OLD
# print(save_dir_old)

In [ ]:
# ### OLD
# started_tm = datetime.now().strftime('%Y%m%d%H%M%S')
# file_nm_old1 = 'crawler_{}_part1_{}.csv'.format(bbs_sn_num,started_tm)

In [ ]:
# ### OLD
# df_out.to_csv(os.path.join(save_dir_old,file_nm_old1),index=False)

### Preprocessing (new)

In [17]:
### NEW
len_lst = [len(url_pg_lst), len(url_lst), len(ttl_lst), len(dt_lst), len(cntnt_lst), len(cprt_lst)]
print(len_lst)

In [15]:
### NEW
if len(url_pg_lst) == len(url_lst) == len(ttl_lst) == len(dt_lst) == len(cntnt_lst) == len(cprt_lst):
    df_out = pd.DataFrame(np.column_stack([url_pg_lst, url_lst, ttl_lst, dt_lst, cntnt_lst, cprt_lst]),
                      columns=['ATC_LIST_URL', 'ATC_URL', 'ATC_TITLE', 'ENROLL_DATE', 'ATC_CONTENTS', 'RGT_TYPE_CODE_'])
else:
    num_limit = min(len_lst)
    df_out = pd.DataFrame(np.column_stack([url_pg_lst[:num_limit], url_lst[:num_limit], ttl_lst[:num_limit], dt_lst[:num_limit], cntnt_lst[:num_limit], cprt_lst[:num_limit]]),
                      columns=['ATC_LIST_URL', 'ATC_URL', 'ATC_TITLE', 'ENROLL_DATE', 'ATC_CONTENTS', 'RGT_TYPE_CODE_'])

bbbbb


In [85]:
preprocess_xlsx(df_out)

In [ ]:
pdi_reformer(df_out)
pdi_reformer2(df_out)
pdi_reformer3(df_out)
pdi_reformer4(df_out)

### Saving the data (new)

In [ ]:
### NEW
folder_nm = 'EXCEL'
dir_main = os.getcwd()
os.makedirs(folder_nm, exist_ok=True)
save_dir = os.path.join(dir_main, folder_nm)
started_tm = datetime.now().strftime('%Y%m%d%H%M%S')
file_nm = 'work_{}.xlsx'.format(started_tm)

In [ ]:
### NEW
### 엑셀파일 저장
df_out.to_excel(os.path.join(save_dir,file_nm))

### 'CN_LINK_ADDR' (CLA) (old)

In [ ]:
# # 페이지 별로 'CN_LINK_ADDR'들을 한 꺼번에 구함 # 예 1페이지, 10개의 PCLA
# # 추후 필요시 커스텀 바람
# pcla_lst2_ = []
# for i in range(tot_page):
#     url_input_ = str(url_str_.format(start_page + i))
#     wpage = requests.get(url_input_, verify=False)
#     html_enc = wpage.text.encode('utf-8')
#     soup_ = BeautifulSoup(html_enc, 'html.parser')
#     # select : 한꺼번에 선택
#     ul = soup_.select(pcla_css)
#     # 선택된 <a href ...> 태그가 1개 이상일 때
#     if len(ul) != 0:
#         # 일반적인 경우
#         cnt_a_ = len(ul)
#         # 예외처리
#         # 마지막 페이지, 수집 페이지 개수 2개 이상, 마지막 글 개수 0이 아님
#         if i == tot_page - 1 and tot_page > 1 and num_a_fin != 0:
#             cnt_a_ = num_a_fin
#         # 페이지 개수 1개이고 모으는 글의 개수가 한 페이지에 있어서 글 최대 개수 이하일 때
#         if tot_page == 1 and num_a_fin != 0 and num_a_fin <= len(ul):
#             cnt_a_ = num_a_fin
#         # 페이지 개수 0개(start, end 같은 페이지)인데 모으는 글의 개수가 0개 이면 수집 취소
#         if tot_page == 0 and num_a_fin == 0:
#             print('수집할 데이터가 없습니다.')
#             pcla_lst2_.append('')
#             break
#         for j in range(cnt_a_):
#             #print(str(ul[j]))
#             #print('*'*20)
#             pcla_lst2_.append(str(ul[j]))
#     ### -------------------------------------------------------------------------------------------------
#     print('{} 번째 페이지 수집 완료'.format(i + start_page))

In [ ]:
# ### OLD
# df_out2 = pd.DataFrame({'CN_LINK_ADDR': pcla_lst2_})

In [ ]:
# ### OLD
# started_tm = datetime.now().strftime('%Y%m%d%H%M%S')
# file_nm_old2 = 'crawler_{}_part2_{}.csv'.format(bbs_sn_num,started_tm)

In [ ]:
# ### OLD
# df_out2.to_csv(os.path.join(save_dir_old,file_nm_old2),index=False)

### Integrity Test (Old)

In [ ]:
### 정합성 체크 구현 필요...

In [91]:
len(df_out), len(df_out2)

(2, 2)

In [92]:
if len(df_out) == len(df_out2):
    # 서로 합치기
    df_out3 = pd.concat([df_out, df_out2], axis=1)
    unnamed_cols = [col for col in df_out3.columns if 'Unnamed' in col]
    df_out3.drop(unnamed_cols, axis='columns', inplace=True)
else:
    # 정합성 안 맞을 경우 각각의 데이터를 파일로 저장
    print('정합성이 안 맞습니다. 코드 관리자에게 문의주세요.')
    df_out.to_csv(save_dir + 'df_crawler_{}_part1_{}.csv'.format(bbs_sn_num,started_tm),index=False)
    df_out2.to_csv(save_dir + 'df_crawler_{}_part2_{}.csv'.format(bbs_sn_num,started_tm),index=False)

### Preprocessing 2 (old)

In [93]:
# df_out3_ = puvi_prprc(df_out3)
# df_out4 = col_organizer(df_out3_)

### Saving the data (old)

In [94]:
# # CLCT_INFO용 데이터 저장
# df_out4.to_csv(save_dir + "df_clct_info_{}_{}.csv".format(bbs_sn_num,started_tm),index=False)
# print("'CLCT_INFO'의 데이터 수집이 완료되었습니다.")

'CLCT_INFO'의 데이터 수집이 완료되었습니다.


In [95]:
# df_out5 = df_out4.copy(deep=True)
# df_out5_ = ci_dtl_maker(df_out5)
# df_out6 = col_organizer(df_out5_)

In [96]:
# # CLCT_INFO_DTL용 데이터 저장
# df_out6.to_csv(save_dir + "df_clct_info_dtl_{}_{}.csv".format(bbs_sn_num,started_tm),index=False)
# print("'CLCT_INFO_DTL'의 데이터 수집이 완료되었습니다.")

'CLCT_INFO_DTL'의 데이터 수집이 완료되었습니다.


In [97]:
# print('수고하셨습니다.')
# print('다른 게시판 작업이 남았을 경우, 다음 게시판으로 넘어가주세요.')

수고하셨습니다.
다른 게시판 작업이 남았을 경우, 다음 게시판으로 넘어가주세요.
